In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
import numpy as np
import matplotlib.pyplot as plt
from feature_manager import FeatureManager
from dnn_classifier import DNNClassifer
from random import randint
from visualizer import *

In [ ]:
data = pd.read_csv("../data/BTCUSDT-1d.csv", parse_dates=["Open Time"], index_col = "Open Time")
granular_data = pd.read_csv("../data/BTCUSDT-15m.csv", parse_dates=["Open Time"], index_col = "Open Time")
fm = FeatureManager()
fm.build_feature(data=data,lags=0)

In [ ]:
def first_time_go_above_price(start_time,end_time,target_price):
    ''' 
    Find the time in ms that take profit and stop loss condition satisified in the granular data for short trade
    Return time in unix utc milisecond.
    '''
    lookup_data = granular_data.iloc[
        (granular_data.index>=start_time) & 
        (granular_data.index<end_time)].copy()

    first_index = lookup_data.loc[lookup_data["High"]>=target_price].index.min()
    return first_index

def first_time_go_below_price(start_time,end_time,target_price):
    ''' 
    Find the time in ms that take profit and stop loss condition satisified in the granular data for short trade
    Return time in unix utc milisecond.
    '''
    lookup_data = granular_data.iloc[
        (granular_data.index>=start_time) & 
        (granular_data.index<end_time)].copy()

    first_index = lookup_data.loc[lookup_data["Low"]<target_price].index.min()
    return first_index

In [ ]:
def calculate_tp_or_sl(row,i,is_long,timeframe_in_ms = 24*60*60*1000):    
    '''Calculate if a take profit, stop loss event happen
    '''
    
    if is_long:
        open_cond = row["long_decision_forward_{}".format(i-1)]==0
        tp_cond = row["High_forward_{}".format(i)] >= row["long_take_profit"]
        sl_cond = row["Low_forward_{}".format(i)] <= row["long_stop_loss"]

    else:
        open_cond = row["short_decision_forward_{}".format(i-1)]==0
        tp_cond = row["Low_forward_{}".format(i)] <= row["short_take_profit"]
        sl_cond = row["High_forward_{}".format(i)] >= row["short_stop_loss"]
    
        
    if open_cond:
        if tp_cond:                                         #  If take-profit hitted
            if sl_cond:
                if is_long:
                    first_tp_index = first_time_go_above_price(
                        start_time=int(row.name) + i * timeframe_in_ms,
                        end_time=int(row.name) + (i+1)* timeframe_in_ms,
                        target_price = row["long_take_profit"],
                    )
                    first_sl_index = first_time_go_below_price(
                        start_time=int(row.name) + i * timeframe_in_ms,
                        end_time=int(row.name) + (i+1)* timeframe_in_ms,
                        target_price = row["long_stop_loss"],
                    )
                else:
                    first_tp_index = first_time_go_below_price(
                        start_time=int(row.name) + i * timeframe_in_ms,
                        end_time=int(row.name) + (i+1)* timeframe_in_ms,
                        target_price = row["short_take_profit"],
                    )
                    first_sl_index = first_time_go_above_price(
                        start_time=int(row.name) + i * timeframe_in_ms,
                        end_time=int(row.name) + (i+1)* timeframe_in_ms,
                        target_price = row["short_stop_loss"],
                    )
                if first_tp_index < first_sl_index:         #  Granular says it is take-profit
                    return [True, False]
                elif first_tp_index > first_sl_index:       #   Granular says it is take-profit
                    return [False, True]
                else:                                       #   Granular can not tell, use random
                    ran_bool = (randint(0,99) < 50)
                    return [ran_bool, ~ran_bool]
            
            else:   # stop-loss not hitted
                return [True, False]
        else:
            if sl_cond: #if only stop-loss hitted
                return [False, True]
            else:           #if both not hitted
                return [False, False]
    else:
        return [False, False]

In [ ]:
def prepare_trade_forward_data(data, take_profit_rate = 0.05, stop_loss_rate = 0.025, max_duration = 7):
    '''   
    # Compute future will happen outcome and and trade signal
    # Will update long_decision_forward to 1: take profit, -1: stop loss, 0: keep open, 2: closed 
    # Will update trade_signal to 1: long, 2: short, 0: no trade
    '''

    data["long_take_profit"] = data["Close"]*(1+take_profit_rate)
    data["long_stop_loss"] = data["Close"]*(1-stop_loss_rate)
    data["short_take_profit"] = data["Close"]*(1-take_profit_rate)
    data["short_stop_loss"] = data["Close"]*(1+stop_loss_rate)
    data["long_decision_forward_0"] = 0
    data["short_decision_forward_0"] = 0
    trade_signal_str = "trade_signal"
    trade_return_str = "trade_return"
    data[trade_signal_str] = 0
    data[trade_return_str] = 0
    
    for i in range(1,max_duration+1):
        
        data["High_forward_{}".format(i)] = data["High"].shift(-i)
        data["Low_forward_{}".format(i)] = data["Low"].shift(-i)
   
        long_str = "long_decision_forward_{}".format(i)
        short_str = "short_decision_forward_{}".format(i)

        #Temporarily set all open as closed
        data[long_str] = 2
        data[short_str] = 2

        open_long_cond = data["long_decision_forward_{}".format(i-1)]==0
        open_short_condition = data["short_decision_forward_{}".format(i-1)]==0

        # Compute decision to be take profit or stop loss
        data[["ol_tp","ol_sl"]]=data.apply(lambda row: calculate_tp_or_sl(
            row=row,
            i=i,
            is_long=True
            ),
            result_type = "expand",
            axis=1
        )

        ol_tp_cond = data["ol_tp"]
        ol_sl_cond = data["ol_sl"]
        

        # Open long positions not hit either take-profit or stop-loss, leave it open
        data.loc[open_long_cond & ~ol_tp_cond & ~ol_sl_cond, long_str] = 0
        
        # Open long positions hit stop-loss
        data.loc[open_long_cond & ol_sl_cond & ~ol_tp_cond,long_str] = -1

        # Open long positions hit take-profit
        data.loc[open_long_cond & ol_tp_cond & ~ol_sl_cond,long_str] = 1
        data.loc[open_long_cond & ol_tp_cond & ~ol_sl_cond,trade_signal_str] = 1
        data.loc[open_long_cond & ol_tp_cond & ~ol_sl_cond,trade_return_str] = np.log(1 + take_profit_rate)      
            
        # Compute future outcome for open short positions
        data[["os_tp","os_sl"]] = data.apply(lambda row: calculate_tp_or_sl(
            row=row,
            i=i,
            is_long=False
            ),
            result_type = "expand",
            axis=1
        )
        os_tp_cond = data["os_tp"]
        os_sl_cond = data["os_sl"]
        
        # Open short positions not hit either take-profit or stop-loss, leave it open
        data.loc[open_short_condition & ~os_tp_cond & ~os_sl_cond, short_str] = 0

        # Open short positions hit stop-loss
        data.loc[open_short_condition & os_sl_cond & ~os_tp_cond,short_str] = -1

        # Open short positions hit take-profit
        data.loc[open_short_condition & os_tp_cond & ~os_sl_cond, short_str] = 1
        data.loc[open_short_condition & os_tp_cond & ~os_sl_cond,trade_signal_str] = 2
        data.loc[open_short_condition & os_tp_cond & ~os_sl_cond,trade_return_str] = np.log(1 + take_profit_rate)
        
       

In [ ]:
prepare_trade_forward_data(data = fm.df,take_profit_rate=0.04,stop_loss_rate=0.02)

In [ ]:
fm.df.trade_signal.value_counts()